In [1]:
#!pip install --upgrade pika

In [2]:
from billiard import current_process
import time
import math
import socket
import numpy as np
import os
import random

In [3]:
import pika
pika.__version__

'1.1.0'

In [4]:
#!sudo rabbitmqctl add_user myguest myguestpwd
#!sudo rabbitmqctl set_permissions -p / myguest "." "." ".*"

In [5]:
!sudo rabbitmqctl list_users

Listing users ...
guest	[administrator]
myguest	[]


In [6]:
!sudo rabbitmqctl list_bindings

Listing bindings ...
	exchange	amq.gen-vbQdJOYlGUaM4EhbdctsLQ	queue	amq.gen-vbQdJOYlGUaM4EhbdctsLQ	[]
direct_pings	exchange	amq.gen-vbQdJOYlGUaM4EhbdctsLQ	queue	rpc_queue	[]


In [7]:
!sudo rabbitmqctl list_exchanges

Listing exchanges ...
	direct
amq.direct	direct
amq.fanout	fanout
amq.headers	headers
amq.match	headers
amq.rabbitmq.log	topic
amq.rabbitmq.trace	topic
amq.topic	topic
direct_pings	direct


In [8]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
#URL='localhost'
URL='PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com'
parameters =  pika.ConnectionParameters(URL, port=5672, credentials=credentials)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()
exchangeName = 'direct_pings'

In [9]:
def calculate_message(msg):
    
    process_name=current_process()._name
    try:
        process_index=str(current_process().index)
    except:
        process_index='_none_'
    
    tmpip= socket.gethostbyname(socket.gethostname())
    
    results=tmpip+' process_name: '+process_name+' process_index: '+process_index+' os_pid: '+str(os.getpid())+' time: '+str(time.time())+' message: '+str(msg)
    
    #time.sleep(random.uniform(0,7)) # Let us wait some random time to simulate that some jobs might take longer time
    
    return results

    #if n == 0:
    #    return 0
    #elif n == 1:
    #    return 1
    #else:
    #    return fib(n - 1) + fib(n - 2)

In [10]:
def on_request(ch, method, props, body):
    msg = str(body) # body is the message to the worker. We want to calculate Fib(n)

    print(" I just received message:", msg)
    
    print('ch:', ch)
    print('method:', method)
    print('props:', props)
    
    print('props.reply_to:', props.reply_to)
    print('props.correlation_id:', props.correlation_id)
    
    response = calculate_message(msg) # Results, Fib(n), calculated
    
    # We need to send back the results to the client
    ch.basic_publish(exchange='', # Default exchange
                     routing_key=props.reply_to, # This is the routing key to the queue that we use for sending the answer
                     #use the same correlation_id so the client knows that this is a reply to what message
                     properties=pika.BasicProperties(correlation_id = props.correlation_id), 
                     body=str(response))
    #ch.basic_ack(delivery_tag=method.delivery_tag)
    print(" Calculations done. Results sent back to the server")
    print(" *** \n")


In [11]:
channel.exchange_declare(exchange=exchangeName, exchange_type='direct')

result = channel.queue_declare(queue='', exclusive=True)
queue_name = result.method.queue

channel.queue_bind(exchange=exchangeName, queue=queue_name,routing_key='rpc_queue')

#channel.basic_qos(prefetch_count=1)


<METHOD(['channel_number=1', 'frame_type=1', 'method=<Queue.BindOk>'])>

In [12]:
#When the worker receives a message it will tun the "on_request" call back function
channel.basic_consume(queue=queue_name, on_message_callback=on_request, auto_ack=True)  

'ctag1.7b0dfdae7e954913b5800614b30605b9'

In [ ]:
print(" [x] Awaiting RPC requests")
channel.start_consuming()

 [x] Awaiting RPC requests
 I just received message: b'hi'
ch: <BlockingChannel impl=<Channel number=1 OPEN conn=<SelectConnection OPEN transport=<pika.adapters.utils.io_services_utils._AsyncPlaintextTransport object at 0x7ff67c05d6d8> params=<ConnectionParameters host=PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com port=5672 virtual_host=/ ssl=False>>>>
method: <Basic.Deliver(['consumer_tag=ctag1.7b0dfdae7e954913b5800614b30605b9', 'delivery_tag=1', 'exchange=direct_pings', 'redelivered=False', 'routing_key=rpc_queue'])>
props: <BasicProperties(['content_type=text/plain', 'correlation_id=10d9746b-ac5c-459b-9c02-dc7289931465', 'delivery_mode=1', 'reply_to=amq.gen-6-3BnoIcWTRp3sjv1Fhm6w'])>
props.reply_to: amq.gen-6-3BnoIcWTRp3sjv1Fhm6w
props.correlation_id: 10d9746b-ac5c-459b-9c02-dc7289931465
 Calculations done. Results sent back to the server
 *** 

 I just received message: b'huh'
ch: <BlockingChannel impl=<Channel number=1 OPEN conn=<SelectConnection OPEN transport=<pika.adapt

 I just received message: b'hey :)'
ch: <BlockingChannel impl=<Channel number=1 OPEN conn=<SelectConnection OPEN transport=<pika.adapters.utils.io_services_utils._AsyncPlaintextTransport object at 0x7ff67c05d6d8> params=<ConnectionParameters host=PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com port=5672 virtual_host=/ ssl=False>>>>
method: <Basic.Deliver(['consumer_tag=ctag1.7b0dfdae7e954913b5800614b30605b9', 'delivery_tag=11', 'exchange=direct_pings', 'redelivered=False', 'routing_key=rpc_queue'])>
props: <BasicProperties(['content_type=text/plain', 'correlation_id=373c5421-6857-408b-bd0d-bdd99827fe5d', 'delivery_mode=1', 'reply_to=amq.gen-6-3BnoIcWTRp3sjv1Fhm6w'])>
props.reply_to: amq.gen-6-3BnoIcWTRp3sjv1Fhm6w
props.correlation_id: 373c5421-6857-408b-bd0d-bdd99827fe5d
 Calculations done. Results sent back to the server
 *** 

 I just received message: b'lalala'
ch: <BlockingChannel impl=<Channel number=1 OPEN conn=<SelectConnection OPEN transport=<pika.adapters.utils.io_servic

 I just received message: b'lalala'
ch: <BlockingChannel impl=<Channel number=1 OPEN conn=<SelectConnection OPEN transport=<pika.adapters.utils.io_services_utils._AsyncPlaintextTransport object at 0x7ff67c05d6d8> params=<ConnectionParameters host=PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com port=5672 virtual_host=/ ssl=False>>>>
method: <Basic.Deliver(['consumer_tag=ctag1.7b0dfdae7e954913b5800614b30605b9', 'delivery_tag=21', 'exchange=direct_pings', 'redelivered=False', 'routing_key=rpc_queue'])>
props: <BasicProperties(['content_type=text/plain', 'correlation_id=529d18f7-9706-4fae-a8e1-eb5df2e0f8c6', 'delivery_mode=1', 'reply_to=amq.gen-Sv7AtFiRX0s0VTcJrSCLMQ'])>
props.reply_to: amq.gen-Sv7AtFiRX0s0VTcJrSCLMQ
props.correlation_id: 529d18f7-9706-4fae-a8e1-eb5df2e0f8c6
 Calculations done. Results sent back to the server
 *** 

 I just received message: b'hmm'
ch: <BlockingChannel impl=<Channel number=1 OPEN conn=<SelectConnection OPEN transport=<pika.adapters.utils.io_services_